In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy import ndimage
from func import *
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from matplotlib.colors import LogNorm

In [2]:
def err_ratio(data,err):   
    (y_blr,x_blr) = ndimage.measurements.maximum_position(data)
    (act_dat,act_err) = (data,err)
    data_mask = data[2:-2,2:-2]
    err_mask = err[2:-2,2:-2]
    data[2:-2,2:-2]=0
    err[2:-2,2:-2]=0
    data_edge = data[data!=0]
    err_edge = err[err!=0]
    err_2D_dat = np.std(data_edge)
    err_2D_err = np.mean(err_edge)
    correct_fact = np.abs(err_2D_dat/err_2D_err)
    return correct_fact
    
def total_flux_and_err(Hb1_blr_br_data,Hb2_blr_br_data,OIII_br_data,OIII_nr_data,vel_sigma_Hb1,vel_sigma_Hb2,vel_sigma_OIII_br,vel_sigma_OIII,Hb1_blr_br_err,Hb2_blr_br_err,OIII_br_err,OIII_nr_err,vel_sigma_Hb1_err,vel_sigma_Hb2_err,vel_sigma_OIII_br_err,vel_sigma_OIII_err,broad2=False,MC_loops=100):
    sigma_eff_Hb1 = line_width_recons(vel_sigma_Hb1,rest_line=4861.33,inst_res_fwhm=0)
    sigma_eff_Hb2 = line_width_recons(vel_sigma_Hb2,rest_line=4861.33,inst_res_fwhm=0) 
    sigma_eff_OIII_br = line_width_recons(vel_sigma_OIII_br,rest_line=5006.8,inst_res_fwhm=0) 
    sigma_eff_OIII_nr = line_width_recons(vel_sigma_OIII,rest_line=5006.8,inst_res_fwhm=0) 
    
    flux_Hb1 = Hb1_blr_br_data*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux_Hb2 = Hb2_blr_br_data*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2))   
    
    if broad2:
        flux_Hb_blr = flux_Hb1+flux_Hb2
    else:
        flux_Hb_blr = flux_Hb1
   
    flux_OIII_br = OIII_br_data*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_OIII_br))
    flux_OIII_nr = OIII_nr_data*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_OIII_nr))  
    flux_par = [flux_Hb1,flux_Hb2,flux_Hb_blr,flux_OIII_br,flux_OIII_nr]
    
    flux_err = np.zeros((len(flux_par),flux_Hb_blr.shape[0],flux_Hb_blr.shape[1]),dtype=np.float32)

    for i in range (flux_Hb_blr.shape[0]):
        for j in range (flux_Hb_blr.shape[1]):
            (Hb1_blr_br_pix_data,Hb2_blr_br_pix_data,OIII_br_pix_data,OIII_nr_pix_data) = (Hb1_blr_br_data[i,j],Hb2_blr_br_data[i,j],OIII_br_data[i,j],OIII_nr_data[i,j])
            (Hb1_blr_br_pix_err,Hb2_blr_br_pix_err,OIII_br_pix_err,OIII_nr_pix_err) = (Hb1_blr_br_err[i,j],Hb2_blr_br_err[i,j],OIII_br_err[i,j],OIII_nr_err[i,j])

            MC_par = np.zeros((len(flux_par),MC_loops))
            for l in range (MC_loops):
                Hb1_blr_br_pix_data_iter = np.random.normal(Hb1_blr_br_pix_data,Hb1_blr_br_pix_err)
                Hb2_blr_br_pix_data_iter = np.random.normal(Hb2_blr_br_pix_data,Hb2_blr_br_pix_err)
                OIII_br_pix_data_iter = np.random.normal(OIII_br_pix_data,OIII_br_pix_err)
                OIII_nr_pix_data_iter = np.random.normal(OIII_nr_pix_data,OIII_nr_pix_err)
            
                vel_sigma_Hb1_iter = np.random.normal(vel_sigma_Hb1,vel_sigma_Hb1_err)
                vel_sigma_Hb2_iter = np.random.normal(vel_sigma_Hb2,vel_sigma_Hb2_err)
                vel_sigma_OIII_br_iter = np.random.normal(vel_sigma_OIII_br,vel_sigma_OIII_br_err)
                vel_sigma_OIII_nr_iter = np.random.normal(vel_sigma_OIII,vel_sigma_OIII_err)
 
                sigma_eff_Hb1_iter = line_width_recons(vel_sigma_Hb1_iter,rest_line=4861.33,inst_res_fwhm=0)
                sigma_eff_Hb2_iter = line_width_recons(vel_sigma_Hb2_iter,rest_line=4861.33,inst_res_fwhm=0)
                sigma_eff_OIII_br_iter = line_width_recons(vel_sigma_OIII_br_iter,rest_line=5006.8,inst_res_fwhm=0) 
                sigma_eff_OIII_nr_iter = line_width_recons(vel_sigma_OIII_nr_iter,rest_line=5006.8,inst_res_fwhm=0) 

                flux_Hb1_pix_data_MC = Hb1_blr_br_pix_data_iter*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1_iter))
                flux_Hb2_pix_data_MC = Hb2_blr_br_pix_data_iter*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2_iter))
    
                if broad2:
                    flux_Hb_blr_pix_data_MC = flux_Hb1_pix_data_MC+flux_Hb2_pix_data_MC
                else:
                    flux_Hb_blr_pix_data_MC = flux_Hb1_pix_data_MC
           
                flux_OIII_br_pix_data_MC = OIII_br_pix_data_iter*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_OIII_br_iter))
                flux_OIII_nr_pix_data_MC = OIII_nr_pix_data_iter*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_OIII_nr_iter))
                
                flux_array_MC = [flux_Hb1_pix_data_MC,flux_Hb2_pix_data_MC,flux_Hb_blr_pix_data_MC,flux_OIII_br_pix_data_MC,flux_OIII_nr_pix_data_MC]
                MC_par[:,l] = flux_array_MC
            flux_pix_err = np.std(MC_par,1)
            flux_err[:,i,j] = flux_pix_err
    return flux_par,flux_err


In [3]:
def total_flux_map(output_par,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    [flux_Hb1,flux_Hb2,flux_Hb_blr,flux_OIII_br,flux_OIII_nr] = output_par
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(flux_Hb1,name='flux_Hb1_blr'))
    hdus.append(fits.ImageHDU(flux_Hb2,name='flux_Hb2_blr'))
    hdus.append(fits.ImageHDU(flux_Hb_blr,name='flux_Hb_blr'))
    hdus.append(fits.ImageHDU(flux_OIII_br,name='flux_OIII_br'))
    hdus.append(fits.ImageHDU(flux_OIII_nr,name='flux_OIII_nr'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/9_arcsec_fluxcube_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def total_flux_err_map(output_par_err,obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    [flux_Hb1_err,flux_Hb2_err,flux_Hb_blr_err,flux_OIII_br_err,flux_OIII_nr_err] = output_par_err
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(flux_Hb1_err,name='flux_Hb1_blr_err'))
    hdus.append(fits.ImageHDU(flux_Hb2_err,name='flux_Hb2_blr_err'))
    hdus.append(fits.ImageHDU(flux_Hb_blr_err,name='flux_Hb_blr_err'))
    hdus.append(fits.ImageHDU(flux_OIII_br_err,name='flux_OIII_br_err'))
    hdus.append(fits.ImageHDU(flux_OIII_nr_err,name='flux_OIII_nr_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/9_arcsec_fluxcube_err_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')

def new_flux_err_map(obj,blr_correction_fact,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    hdu = fits.open('%s/Flux Maps/%s/9_arcsec_fluxcube_err_%s.fits'%(destination_path_cube,obj,obj))
    Hb1_error = hdu[1].data
    Hb2_error = hdu[2].data
    Hb_error = hdu[3].data
    OIII_br_error = hdu[4].data
    OIII_nr_error = hdu[5].data
    Hb1_err_new = blr_correction_fact*Hb1_error
    Hb2_err_new = blr_correction_fact*Hb2_error
    Hb_err_new = blr_correction_fact*Hb_error
    OIII_br_err_new = blr_correction_fact*OIII_br_error
    OIII_nr_err_new = blr_correction_fact*OIII_nr_error
    hdus=[]
    hdus.append(fits.PrimaryHDU())
    hdus.append(fits.ImageHDU(Hb1_err_new,name='flux_Hb1_blr_err'))
    hdus.append(fits.ImageHDU(Hb2_err_new,name='flux_Hb2_blr_err'))
    hdus.append(fits.ImageHDU(Hb_err_new,name='flux_Hb_blr_err'))
    hdus.append(fits.ImageHDU(OIII_br_err_new,name='flux_OIII_br_err'))
    hdus.append(fits.ImageHDU(OIII_nr_err_new,name='flux_OIII_nr_err'))
    hdu = fits.HDUList(hdus)
    hdu.writeto('%s/Flux Maps/%s/9_arcsec_fluxcube_err_%s.fits'%(destination_path_cube,obj,obj),overwrite='True')


In [4]:
def algorithm_script(obj,z,broad2,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    print ('%s'%(obj))

    (amp_Hb1,amp_Hb2,vel_Hb1,vel_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset,m,c) = central_par(obj)
    (vel_sigma_Hb1_err,vel_sigma_Hb2_err,vel_sigma_OIII_err,vel_sigma_OIII_br_err) = central_vel_sigma_err(obj)
    
    (Hb1_blr_br_data,Hb2_blr_br_data) = loadblr('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj))
    (Hb1_blr_br_err,Hb2_blr_br_err) = loadblr('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_err = loadwing('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_br_data = loadwing('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj)) # It opens the hdu[3] where the OIII_br data 
    OIII_nr_err = loadcore('%s/Flux Maps/%s/9_arcsec_subcube_par_err_%s.fits'%(destination_path_cube,obj,obj))
    OIII_nr_data = loadcore('%s/Flux Maps/%s/9_arcsec_subcube_par_%s.fits'%(destination_path_cube,obj,obj)) # It opens the hdu[3] where the OIII_br data 
    
    (output_par,output_par_err) = total_flux_and_err(Hb1_blr_br_data,Hb2_blr_br_data,OIII_br_data,OIII_nr_data,vel_sigma_Hb1,vel_sigma_Hb2,vel_sigma_OIII_br,vel_sigma_OIII,Hb1_blr_br_err,Hb2_blr_br_err,OIII_br_err,OIII_nr_err,vel_sigma_Hb1_err,vel_sigma_Hb2_err,vel_sigma_OIII_br_err,vel_sigma_OIII_err,broad2,MC_loops=100)
    [flux_Hb1,flux_Hb2,flux_Hb_blr,flux_OIII_br,flux_OIII_nr] = output_par
    [flux_Hb1_err,flux_Hb2_err,flux_Hb_blr_err,flux_OIII_br_err,flux_OIII_nr_err] = output_par_err

    total_flux_map(output_par,obj)   
    total_flux_err_map(output_par_err,obj)
    blr_err_correction_fact = err_ratio(flux_Hb_blr,flux_Hb_blr_err)
    print (blr_err_correction_fact)
    new_flux_err_map(obj,blr_err_correction_fact)
 

In [5]:
z = {"HE0021-1819":0.053197}

broad2= {'HE0021-1819':False}

objs = z.keys()


for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE0021-1819
1.5795586


In [6]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':False}

objs = z.keys()


for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

HE0021-1810
1.1109729
HE0021-1819
1.5936551
HE0040-1105
1.6535398
HE0108-4743
1.0021734
HE0114-0015
1.645223
HE0119-0118
1.0367256
HE0212-0059
1.3627026
HE0224-2834
1.7529925
HE0227-0913
1.5613725
HE0232-0900
1.8808694
HE0253-1641
1.4330839
HE0345+0056
1.5112928
HE0351+0240
1.208011
HE0412-0803
1.4327191
HE0429-0247
0.86651057
HE0433-1028
0.52522147
HE0853+0102
0.91898537
HE0934+0119
0.9288791
HE1011-0403
0.66657853
HE1017-0305
1.5650632
HE1029-1831
0.96980655
HE1107-0813
0.7430816
HE1108-2813
1.8394359
HE1126-0407
1.0900819
HE1237-0504
2.2802892
HE1248-1356
1.719528
HE1330-1013
1.1411827
HE1353-1917
1.526466
HE1417-0909
1.4705609
HE2128-0221
1.6034054
HE2211-3903
1.9088222
HE2222-0026
1.1417463
HE2233+0124
1.3162339
HE2302-0857
2.0456107


In [8]:
a = [1,2,3]
b = [4,5,6]
np.multiply(a,b)

array([ 4, 10, 18])

In [31]:
hdu = fits.open('subcube_par_HE1353-1917.fits')
OIII_nr_data = hdu[2].data
hdu.close()

hdu = fits.open('subcube_par_err_HE1353-1917.fits')
OIII_nr_err = hdu[2].data
hdu.close()

In [41]:
core_err = np.zeros((1,OIII_nr_data.shape[0],OIII_nr_data.shape[1]),dtype=np.float32)
for i in range (OIII_nr_data.shape[0]):
    for j in range (OIII_nr_data.shape[1]):
        data = OIII_nr_data[i,j]
        err = OIII_nr_err[i,j]
        flux_par = np.zeros((1,100))
        for l in range (100):
            core_flux_amp_pix = np.random.normal(data,err)
            flux_par[:,l] = core_flux_amp_pix
        flux_par_err = np.std(flux_par,1)
        core_err[:,i,j] = flux_par_err

print (np.shape(core_err))

(1, 17, 17)
